# Adding Change Point Annotations to Database

### Table doc_string convention:

    class TableName(dj.TableType):
        definition = """
        # Description of Table purpose
        -> Dependency to existing Table           # Purpose of said existing Table
        primary_key: datatype                     # Purpose of primary key 
        ---
        normal_column:  datatype                  # Purpose of column (aka non-primary key) 
        """
        

### (1) Design new Table

In [ ]:
# linking to module with algorithm? 

# dhv_deploy = ("/home/alana/DeepHumanVision_deploy/")
# sys.path.append(dhv_deploy)
# from annotation.data_driven_annotation import cpt as cpt

class ChangePointAnnotation(dj.Computed):
    ### ask: is there a meaning to the breakpoints in the defs? 
    ### as in, above the ---, primary keys,  and below: secondary? 
    """
    # This table contains information about change points detected in spiking acitivity of single units;
    -> BinnedSpikesDuringMovie                    # the unit information and the binned spike vectors
    algorithm: varchar(32)                        # the algorithm version that was used to create the change point
    ---
    change_point: int                             # actual change point in neural recording time
    t_value: int                                  # the t-value of the test statistic
    additional_information="":varchar(32)         # space for additional information
    """
    
    def make(self, key):
        # Retrieve all patient id's and sessions that exist within the 
        # database and use for setting up the new computed table
    
        # binnedsppikesduringmovie is deprecated ; use electrode unit & bin at runtime 
        patient_ids, units_ids, session_nrs, bin_sizes = BinnedSpikesDuringMovie.fetch("patient_id","unit_id", "session_nr", "bin_size")
        
        
        
        
        for i in range(len(patients_ids)):
            pat_id = patient_ids[i]
            unit_nr = units_ids[i]
            unit_bin_size = bin_size[i]
            
            ### ask: what's the best way to do this part within the db_setup portion? 
            ### is referencing the unit in this way not necessary?
            
            # binnedsppikesduringmovie is deprecated 
            binned_unit_name = (BinnedSpikesDuringMovie & "patient_id={}".format(pat_id) 
                           & "unit_id={}".format(unit_nr) & "bin_size={}".format(unit_bin_size)).fetch("spike_vector")[0]
            binned_unit = np.load(binned_unit_name)
            os.remove(binned_unit_name)
            ###
            
            # compute change points
            # note: cpt algo auto-generates plots -- should make default not plotting for this purpose?
            tau, ttest = cpt.find_changepoint_tt(binned_unit)
            
            tstat = ttest[0] 
            
            self.insert1({"patient_id": pat_id, "unit_id": unit_nr,
                          "session_nr": session_nrs[i], "bin_size": unit_bin_size, 
                          "change_point": tau, "t_value": tstat})
            

(2) Define the Type of Table

(3) Initialize Table

(would just be putting in the db_setup file)

(4) Define how the Table with be populated 

(by writing the make function, defines how it would be populated)

(5) Populate the Table

In [ ]:
# insert this line into set_up notebook
ChangePointAnnotation.populate()

# if manual Table, then would need to manually population 
# ExampleTable.insert1(to_be_inserted)
